In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from google.colab import userdata
# from huggingface_hub import login
from huggingface_hub import notebook_login

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar 11 20:25:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install -q datasets pandas pymongo sentence_transformers
!pip install -q -U transformers
# Install below if using GPU
!pip install -q accelerate

Logging in by using "notebook_login" function instead of saving token in userdata environment variable.

In [5]:
notebook_login()

In [6]:
# hf_token = userdata.get('hf_token')

# login(hf_token)

Example Code for Running Gemma 2B Model

In [7]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<bos>Write me a poem about Machine Learning.

Machines, they weave and they learn,
From the data, they discern.
Algorithms, a symphony,
Unleash the power of the machine.

With each iteration, they grow,
A tapestry of insights they sow.
From the past, they learn and they adapt,
A never-ending, ever-fast.

The future holds mysteries untold,
Where machines will shape our world.
They will heal, they will fight, they will play,
A symphony of the digital day.

So let the machines learn and let them soar,
For the future is here, and it's here to stay.
With a touch of humanity, a spark of care,
We embrace the machine, and we share.<eos>


### Vertex and OpenAI code below:

In [ ]:
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
import vertexai

REGION = "us-central1"

PROJECT_ID = "colab-vertex-416920"
# Set the project id
vertexai.init(project=PROJECT_ID, location=REGION)

# A function to generate text embedding.
def text_embedding(
  model_name: str, task_type: str, text: str, title: str = "") -> list:
    """Generate text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained(model_name)

    text_embedding_input = TextEmbeddingInput(
        task_type=task_type, title=title, text=text)
    embeddings = model.get_embeddings([text_embedding_input])
    return embeddings[0].values

# Set the model name.
MODEL_NAME = "textembedding-gecko@002"


TASK_TYPE = "RETRIEVAL_DOCUMENT"
TITLE = "Google"
TEXT = "Embed text."

embedding = text_embedding(model_name=MODEL_NAME, task_type=TASK_TYPE, text=TEXT, title=TITLE)
print(len(embedding))


In [ ]:
!pip install langchain singlestoredb tiktoken openai pypdf
!pip install langchain_openai

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import SingleStoreDB
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
import os

loader = PyPDFLoader("superbowl.pdf")
data = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
key = userdata.get('openai')
# Generate embeddings
embeddings = OpenAIEmbeddings(openai_api_key=key)

# Set up the SingleStoreDB connection and vector database
os.environ[
    "SINGLESTOREDB_URL"] = "ayush:Test1234@svc-3482219c-a389-4079-b18b-d50662524e8a-shared-dml.aws-virginia-6.svc.singlestore.com:3333/database_79fb0"
SingleStoreVectorDB = SingleStoreDB.from_documents(texts, embeddings, table_name="notebook")


In [ ]:

SingleStoreVectorDB = SingleStoreDB.from_documents(texts, embeddings, table_name="notebook")

user_query = "Who won the superbowl"

# Perform similarity search
docs = SingleStoreVectorDB.similarity_search(user_query)
if docs:
    context = docs[0].page_content

    from transformers import AutoTokenizer, AutoModelForCausalLM

    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
    model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

    combined_information = "User Query: " +  user_query + "Context: " + context

    # Moving tensors to GPU
    input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
    response = model.generate(**input_ids, max_new_tokens=500)
    print(tokenizer.decode(response[0]))


else:
    print("AI: Sorry, I couldn't find relevant information.")